In [110]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch.nn as nn
from torch.nn import Sequential
from torch.autograd import Variable

from sklearn.ensemble import RandomForestRegressor

from IPython.display import Markdown, display

## Load Data

In [111]:
df = pd.read_csv(
    "data/plos_fulldata.csv",
    # names=header,
    index_col=0)

/tmp/ipykernel_5731/4005757943.py:1: DtypeWarning: Columns (67,75,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [112]:
display(df)

,filename,doi_x,AccessionID,reading_time,flesch_reading_score,dale_chall_readability_score,total_paragraphs,avg_sentences_per_para,len_of_longest_paragraph,total_sentences,...,cited_by_qna_count,cited_by_peer_review_sites_count,cited_by_weibo_count,cited_by_pinners_count,cited_by_linkedin_count,urn,cited_by_book_reviews_count,editors,attribution,ordinal_number
0,journal.pone.0080347.xml,10.1371/journal.pone.0080347,journal.pone.0080347,29.270,38.69,7.5,36,6.72,21,242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,journal.pone.0054670.xml,10.1371/journal.pone.0054670,journal.pone.0054670,17.565,29.11,7.3,33,4.67,13,154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,journal.pone.0195986.xml,10.1371/journal.pone.0195986,journal.pone.0195986,25.780,26.14,7.7,48,4.98,10,239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,journal.pone.0150364.xml,10.1371/journal.pone.0150364,journal.pone.0150364,25.815,28.30,7.8,43,4.81,10,207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,journal.pone.0184653.xml,10.1371/journal.pone.0184653,journal.pone.0184653,27.640,34.60,7.2,47,5.40,16,254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189843,journal.pone.0212848.xml,10.1371/journal.pone.0212848,journal.pone.0212848,16.840,29.56,7.6,36,3.64,10,131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189844,journal.pone.0157412.xml,10.1371/journal.pone.0157412,journal.pone.0157412,16.545,22.58,6.9,24,6.38,17,153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189845,journal.pone.0109208.xml,10.1371/journal.pone.0109208,journal.pone.0109208,24.015,27.25,7.2,38,6.71,23,255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189846,journal.pntd.0004608.xml,10.1371/journal.pntd.0004608,journal.pntd.0004608,22.815,37.57,7.3,32,5.56,14,178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df.shape

(189848, 79)

In [114]:
df = df.fillna(0)
df = df.sample(frac=1)

In [116]:
display(df)

,filename,doi_x,AccessionID,reading_time,flesch_reading_score,dale_chall_readability_score,total_paragraphs,avg_sentences_per_para,len_of_longest_paragraph,total_sentences,...,cited_by_qna_count,cited_by_peer_review_sites_count,cited_by_weibo_count,cited_by_pinners_count,cited_by_linkedin_count,urn,cited_by_book_reviews_count,editors,attribution,ordinal_number
94505,journal.pone.0040928.xml,10.1371/journal.pone.0040928,journal.pone.0040928,23.735,26.88,7.6,36,5.47,16,197,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
83382,journal.pmed.0020151.xml,10.1371/journal.pmed.0020151,journal.pmed.0020151,6.390,12.57,9.3,20,2.50,5,50,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
90991,journal.pone.0109737.xml,10.1371/journal.pone.0109737,journal.pone.0109737,23.705,32.87,6.8,34,7.03,15,239,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
179233,journal.pntd.0002522.xml,10.1371/journal.pntd.0002522,journal.pntd.0002522,22.000,40.01,6.8,34,5.94,21,202,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
105701,journal.pone.0017867.xml,10.1371/journal.pone.0017867,journal.pone.0017867,30.485,21.26,7.7,41,6.07,13,249,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109779,journal.pone.0207944.xml,10.1371/journal.pone.0207944,journal.pone.0207944,27.480,33.58,7.2,51,5.02,25,256,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
153398,journal.pone.0086528.xml,10.1371/journal.pone.0086528,journal.pone.0086528,13.645,20.35,7.7,25,4.32,12,108,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
93636,journal.ppat.1004893.xml,10.1371/journal.ppat.1004893,journal.ppat.1004893,32.455,34.43,7.3,50,5.30,14,265,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
84232,journal.pone.0051828.xml,10.1371/journal.pone.0051828,journal.pone.0051828,17.670,30.54,7.5,35,4.34,11,152,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0


## Multi-label --> One label

In [117]:
pd.set_option('display.max_colwidth', None)

In [118]:
features = df.iloc[:, 5: 20]
feature_names = df.columns[5: 20]
labels = df[['cited_by_fbwalls_count', 'cited_by_feeds_count',
       'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count',
       'cited_by_tweeters_count', 'cited_by_wikipedia_count',
       'cited_by_accounts_count', 'score']]
features = (features - features.mean()) / features.std()
labels = (labels - labels.mean()) / labels.std()
labels = labels.sum(axis=1)

In [119]:
train_set, validation_set, test_set = np.split(features, [int(0.6 * len(features)), int(0.8 * len(features))])
train_label, validation_label, test_label = np.split(labels, [int(0.6 * len(features)), int(0.8 * len(features))])

### Random Forest Regressor

In [120]:
model = RandomForestRegressor(max_depth=10)
model.fit(train_set, train_label)

RandomForestRegressor(max_depth=10)

In [121]:
score = model.score(train_set, train_label)
print(score)

0.4428386272018838


In [122]:
score = model.score(validation_set, validation_label)
print(score)

0.03363215743585468


### DecisionTreeRegressor

### K Nearest

## One label

In [139]:
features = df.iloc[:, 5: 20]
feature_names = df.columns[5: 20]
labels1 = df[['score']]
features = (features - features.mean()) / features.std()
labels1 = (labels1 - labels1.mean()) / labels1.std()
labels1 = labels1.sum(axis=1)

In [140]:
train_set, validation_set, test_set = np.split(features, [int(0.6 * len(features)), int(0.8 * len(features))])
train_label, validation_label, test_label = np.split(labels, [int(0.6 * len(features)), int(0.8 * len(features))])

### Random Forest Regressor

In [141]:
model = RandomForestRegressor(max_depth=10)
model.fit(train_set, train_label)

RandomForestRegressor(max_depth=10)

In [142]:
score = model.score(train_set, train_label)
print(score)

0.437116745265817


In [143]:
score = model.score(validation_set, validation_label)
print(score)

0.02275082578182186


### DecisionTreeRegressor

### K Nearest